In [1]:
# STL
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
# 3rd Party
import torch
from gatbert.rgat_layer import RGATLayer
torch.manual_seed(0)
from transformers import BertModel, AutoModel

/home/ethanlmines/blue_dir/conda_envs/gatbert/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
in_features = 123
attention_units = 53
out_features = 264
n_heads = 6
n_relations = 7
n_bases = 3
max_nodes = 10
batch_size = 5

In [5]:
gen = torch.Generator().manual_seed(1)
random_features = 5 * (torch.randn(batch_size, max_nodes, in_features, generator=gen) - .5)
random_features.shape

torch.Size([5, 10, 123])

In [6]:
gen = torch.Generator().manual_seed(2)
random_adj = torch.randint(0, 2, size=[batch_size, max_nodes, max_nodes, n_relations], generator=gen).to_sparse()
random_adj

tensor(indices=tensor([[0, 0, 0,  ..., 4, 4, 4],
                       [0, 0, 0,  ..., 9, 9, 9],
                       [0, 0, 0,  ..., 9, 9, 9],
                       [1, 2, 5,  ..., 3, 4, 6]]),
       values=tensor([1, 1, 1,  ..., 1, 1, 1]),
       size=(5, 10, 10, 7), nnz=1779, layout=torch.sparse_coo)

In [8]:
torch.manual_seed(0)
model = RGATLayer(in_features=in_features,
                          attention_units=attention_units,
                          out_features=out_features,
                          n_heads=n_heads,
                          n_relations=n_relations,
                          n_bases=n_bases,
                          attention_mode='argat')

In [9]:
output = model(random_features, random_adj)
output.shape


torch.Size([5, 10, 264])

In [139]:
loss = torch.sum(output)
loss.backward()

Logits grad: tensor([[-1.0378e+00, -6.7509e-01, -2.8978e-01, -6.1378e-01, -2.2774e-01,
          2.9163e-01],
        [-1.7269e+00,  6.4099e-02, -1.3452e+00, -1.7433e-01, -2.3266e+00,
         -5.7234e-01],
        [ 3.0749e+00, -2.2655e+00,  2.4762e-01, -1.6088e+00, -9.8352e-01,
         -1.7201e-01],
        ...,
        [ 6.3667e-01,  6.4911e-01,  7.7401e-04, -3.0992e-01,  1.5062e-01,
         -8.9753e-01],
        [ 2.5477e-01,  6.1890e-01,  1.5265e-01,  8.2389e-02, -5.5311e-01,
          1.5517e-01],
        [-1.2211e-01,  2.1029e-01, -1.0515e-02,  2.1513e-02, -3.0218e-01,
         -4.7483e-01]])
